<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2020-1/Insumos/MertonKMV_Rb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paso de carga de librerías en R de todas las que se utilizarán

In [1]:
install.packages("quantmod")
install.packages("curl")
install.packages("data.table")
install.packages("RCurl")
install.packages("bitops")
install.packages("setInternet2")
install.packages("rootSolve")
install.packages("xml2")
library(rootSolve)
library(quantmod)
library(RCurl)
library(data.table)
library(xml2)
require(jsonlite) #para el getQuotes


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependencies ‘xts’, ‘zoo’, ‘TTR’

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependency ‘bitops’

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
Warning message:
“package ‘setInternet2’ is not available (for R version 3.6.1)”Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeri

In [2]:
Symbols<-c  ("WALMEX.MX")#tienen que ir en orden alfabético
length(Symbols)


start_date=Sys.Date()-3660 #fecha inicial

#Creación del objeto para guardar los datos
dataEnv<-new.env()

#obtener los datos
getSymbols.yahoo(Symbols,env=dataEnv,from=start_date)


#limpiarlos, alinearnos y quedarnos con el precio de cierre 



#muestra de datos
#Nos quedamos con los precios
stock_prices = na.omit(dataEnv$`WALMEX.MX`)
stock_prices=stock_prices[,4] #Close
tail(stock_prices[,])


[1] 1

Warning message:
“WALMEX.MX contains missing values. Some functions will not work if objects contain missing values in the middle of the series. Consider using na.omit(), na.approx(), na.fill(), etc to remove or replace them.”

[1] "WALMEX.MX"

           WALMEX.MX.Close
2019-11-15           54.67
2019-11-19           54.93
2019-11-20           54.54
2019-11-21           53.65
2019-11-22           55.03
2019-12-04           53.90

Se deja el código de "webscrapping" hay que arreglarlo

In [3]:

#PARA CARGAR EL NÚMERO DE ACCIONES EN VENTA

getQuote <- function(ticks) {
  qRoot <- "https://query1.finance.yahoo.com/v7/finance/quote?fields=symbol,longName,regularMarketPrice,regularMarketChange,regularMarketTime,sharesOutstanding,bookValue&formatted=false&symbols="
  z <- fromJSON(paste(qRoot, paste(ticks, collapse=","), sep=""))
  z <- z$quoteResponse$result[,c("symbol", "regularMarketTime", "regularMarketPrice", "regularMarketChange", "longName", "sharesOutstanding", "bookValue" )]
  row.names(z) <- z$symbol
  z$symbol <- NULL
  names(z) <- c("Time", "Price", "Change", "Name", "Numeroacciones", "valorlibros")
  z$Time <- as.POSIXct(z$Time, origin = '1970-01-01 00:00:00')
  return(z)
}


ticks=Symbols

stqt=getQuote(Symbols)


#PARA CARGAR LOS ESTADOS FINANCIEROS Y FLUJOS DE EFECTIVO
url <- "https://finance.yahoo.com/quote/"
scrapy_stocks <- function(stock){
  if ("rvest" %in% installed.packages()) {
    library(rvest)
  }else{
    install.packages("rvest")
    library(rvest)
  }
  for (i in 1:length(stock)) {
    tryCatch(
      {
        # Income Statement
        financial <- paste0(url,stock[i],"/financials?p=",stock[i])
        wahis.session <- html_session(financial)                                
        p <-    wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]/div[2]') %>%
          .[[1]] %>% html_children() %>% html_nodes("div")
        suppressWarnings(Values <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 3] %>% 
                           html_text() %>% gsub(pattern = ",",replacement = "") %>% 
                           gsub(pattern = "-",replacement = 0) %>% as.numeric()) 
        Headers <- wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]') %>% 
          .[[1]] %>% html_nodes("span") %>% html_text() %>% head()
        Concepts <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 2] %>% html_text() %>% .[seq(2,88,4)]
        IS <- data.frame(Breakdown = Concepts[-c(4,16,19)], matrix(Values[!is.na(Values)], ncol = 5, byrow = T))
        names(IS) <- Headers
        temp1 <- IS
        
        # Balance Sheet
        bsheet <- paste0(url,stock[i],"/balance-sheet?p=",stock[i])
        wahis.session <- html_session(bsheet)
        p <-    wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]/div[2]') %>%
          .[[1]] %>% html_children() %>% html_nodes("div")
        suppressWarnings(Values <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 3] %>% 
                           html_text() %>% gsub(pattern = ",",replacement = "") %>% gsub(pattern = "-",replacement = 0) %>% as.numeric()) 
        Headers <- wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]') %>% 
          .[[1]] %>% html_nodes("span") %>% html_text() %>% head(5)
        Concepts <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 2] %>% html_text() %>% .[seq(2,175,4)]
        BS <- data.frame(Breakdown = Concepts[-c(1:3,11,12,22:24,32,39)], matrix(Values[!is.na(Values)], ncol = 4, byrow = T))
        names(BS) <- Headers
        temp2 <- BS
        
        # Cash flows
        cash_f <- paste0(url,stock[i],"/cash-flow?p=",stock[i])
        wahis.session <- html_session(cash_f)
        p <-    wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]/div[2]') %>%
          .[[1]] %>% html_children() %>% html_nodes("div")
        suppressWarnings(Values <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 3] %>% 
                           html_text() %>% gsub(pattern = ",",replacement = "") %>% gsub(pattern = "-",replacement = 0) %>% as.numeric())
        Headers <- wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]') %>% 
          .[[1]] %>% html_nodes("span") %>% html_text() %>% head()
        Concepts <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 2] %>% html_text() %>% .[seq(2,133,4)]
        CF <- data.frame(Breakdown = Concepts[-c(1,13,20,30)], matrix(Values[!is.na(Values)], ncol = 5, byrow = T))
        names(CF) <- Headers
        temp3 <- CF
        
        # Shares Outstanding
        stock_url <- paste0(url,stock[i],"/key-statistics?p=",stock[i])
        SO <- html_session(stock_url) %>%
          html_nodes(xpath = '//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[2]/div/div[2]/div/table/tbody/tr[3]/td[2]') %>% 
          html_text() %>% gsub(pattern = "[A-z]",replacement = "") %>% as.numeric()
        temp4 <- SO
        
        assign(paste0(stock[i],'.f'),value = list(IS = temp1,BS = temp2,CF = temp3, SO = temp4),envir = parent.frame())
        
      },
      error = function(cond){
        message(stock[i], "Give error ",cond)
      }
    )
  }
}

#stock=Symbols
scrapy_stocks(Symbols) 
attach(WALMEX.MX.f)
#WALMEX.MX.f$BS
#WALMEX.MX.f$IS
#WALMEX.MX.f$CF
BS


Breakdown,12/30/2018,12/30/2017,12/30/2016,12/30/2015
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
Cash And Cash Equivalents,38829598,35596117,27975536,24790838
Short Term Investments,0,0,0,0
Total Cash,38829598,35596117,27975536,24790838
Net Receivables,2745233,2414224,2291822,2751227
Inventory,63344265,59463255,53665239,49748874
Other Current Assets,0,982374,0,0
Total Current Assets,115614816,106876206,104022672,84848685
"Gross property, plant and equipment",237056463,223608136,219644234,205815219
Accumulated Depreciation,97688454,88645410,83294631,75592863


In [4]:
TSE <- "Total stockholders' equity"
TL <- "Total Liabilities"
dt <- colnames(BS)[2] # ÚLTIMA FECHA DISPONIBLE


#K_0=BS[33,1]/1000 #Total Stockholder Equity VALOR DE LA EMPRESA
#en millones https://finance.yahoo.com/quote/WALMEX.MX/balance-sheet?p=WALMEX.MX
#Total stockholders' equity
#En millones
K_0=BS[BS$Breakdown==TSE, dt]*1000/1000000
#nrow(BS)
#D_0=BS[24,1]/1000 #Total Liabilities DEUDA ACTUAL
#En millones
D_0=BS[BS$Breakdown==TL, dt]*1000/1000000


#Shares Outstanding (Sexpresa en miles de millones en la página)
#En millones
Numeroacciones=WALMEX.MX.f$SO*1000000000/1000000

dt=strptime(dt, "%m/%d/%Y")
dt=format(dt, "%Y-%m")
dt
#no esta sentencia entonces mi ultima fecha fue tanto 
n=nrow(stock_prices[paste("/",dt,sep="")]) #NUMERO DE FILAS A ANALIZAR PARA EL ÚLTIMO DATO

A_0=stock_prices[paste("/",dt,sep="")][n] #valores actuales

#A_0=A_0*stqt$Numeroacciones/1000000 #VALOR ACTUAL DE LAS ACCIONES VALOR DE ACCIÓN POR ACCIONES DISPONIBLES
A_0=A_0*Numeroacciones #VALOR ACTUAL DE LAS ACCIONES VALOR DE ACCIÓN POR ACCIONES DISPONIBLES
A_0/K_0

A=stock_prices[paste("/",dt,sep="")]  #VECTOR DE PRECIOS 

vol_A=sqrt(var(log(as.matrix(A[2:n])/as.matrix(A[1:(n-1)]))))*sqrt(252) #VOLATILIDAD ANUAL

#Valor del CETE
r=0.0717
T=1

A_0
K_0
vol_A
D_0
r
T


[1] "2018-12"

           WALMEX.MX.Close
2018-12-31         5.29049

           WALMEX.MX.Close
2018-12-31        872476.2

[1] 164914.1

,WALMEX.MX.Close
WALMEX.MX.Close,0.2385214


[1] 141614.8

[1] 0.0717

[1] 1

In [0]:
x=c(D_0,vol_A)

#Función para estimar vol_k y D
G = function(x) 
{
  vol_k <- x[2]
  D <- x[1]
  c(A_0-K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))+D*exp(-r*T)*pnorm((log(K_0/D)+(r-0.5*vol_k^2)*T)/(vol_k*sqrt(T))),
  A_0*vol_A-vol_k*K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T))) )
}

#K_0=164370

ss=multiroot(G,x,positive = TRUE) #RESOLVER EL SISTEMA.

D=ss$root[1]
vol_k=ss$root[2]
ss$root[1]/D_0
vol_A/ss$root[2]
G(c(D,vol_k))

#Probabilidad de incumplimiento
PD_merton=1-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))
PD_merton


Warning message in stode(y, times, func, parms = parms, ...):
“steady-state not reached”

[1] 0

,WALMEX.MX.Close
WALMEX.MX.Close,0.1890184


           WALMEX.MX.Close
2018-12-31        707562.1
2018-12-31             0.0

[1] 0

In [0]:
#Sólo estimar la volatilidad dada la deuda
#Probabilidad de Incumplimiento

Vk = function(x) 
{
  vol_k <- x
  ((vol_k/vol_A)*(K_0/A_0))-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))
}

#Definir una Deuda
D=D_0
x=vol_A #semilla de la vol_k
ss=multiroot(Vk,x,positive = TRUE) #RESOLVER EL SISTEMA.

vol_k=ss$root
print("Ratio vol_k/vol_A")
ss$root[1]/vol_A
print("valor Vk bajo solución")
Vk(vol_k)

PD_merton=1-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))

print("Deuda estimada")
D
print("Deuda sencilla")
D_0
print("Vol estimada")
vol_k
print("Vol semilla")
vol_A
print("PD Merton")
PD_merton



[1] "Ratio vol_k/vol_A"


,WALMEX.MX.Close
WALMEX.MX.Close,4.03426


[1] "valor Vk bajo solución"


           WALMEX.MX.Close
2018-12-31   -9.825218e-11

[1] "Deuda estimada"


...

[1] 0.9645693

[1] "Vol semilla"


,WALMEX.MX.Close
WALMEX.MX.Close,0.2390945


[1] "PD Merton"


[1] 0.2374506

#KMV

In [0]:
print("A_0:")
A_0
print("Valor de K_0")
K_0
print("Valor de vol_A")
vol_A
print("Valor de Deuda Actual")
D_0
print("Tasa libre de riesgo")
r
print("Temporalidad")
T


#Función para estimar vol_k y D
Gkmv = function(x) 
{
  vol_k <- x[2]
  K_0 <- x[1]
  c(A_0-K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))+D*exp(-r*T)*pnorm((log(K_0/D)+(r-0.5*vol_k^2)*T)/(vol_k*sqrt(T))),
    A_0*vol_A-vol_k*K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T))) )
}

D=D_0

x=c(K_0,vol_A)

ss=multiroot(Gkmv,x,positive = TRUE) #RESOLVER EL SISTEMA.

K_e=ss$root[1]
vol_k=ss$root[2]
print("Cociente K_0/D_0")
K_e/D_0
print("Cociente vol_K/vol_A")
ss$root[2]/vol_A
print("Valor de solución en función:")
Gkmv(c(K_e,vol_k))

DI=(K_e-D)/(vol_k*K_0)
print("DI")
DI
EDF=pnorm(-DI)
print("EDF Normal")
EDF
 


[1] "A_0:"


           WALMEX.MX.Close
2018-12-31        872476.2

[1] "Valor de K_0"


[1] 164914.1

[1] "Valor de vol_A"


...

           WALMEX.MX.Close
2018-12-31    2.910383e-11
2018-12-31    0.000000e+00

[1] "DI"


[1] 25.18419

[1] "EDF Normal"


[1] 2.984264e-140